In [1]:
#using PyPlot
include("lib/all.jl");

In [2]:
N=2^25
ax = zeros(N)
ay = zeros(N)
az = zeros(N);

particles = make_particles(N);
t = DTree(particles, 12);

In [3]:
@time fill!(t.tree.nodes, Node());

  0.224079 seconds (3.99 k allocations: 175.127 KB)


In [4]:
@time fill!(t.tree.exps, NodeExp());

  0.250187 seconds (3.88 k allocations: 173.174 KB)


In [5]:
@time group!(t);

In [6]:
@time inform!(t);

  1.019492 seconds (21.83 M allocations: 336.265 MB, 7.39% gc time)


In [7]:
fill!(ax,0.0)
fill!(ay,0.0)
fill!(az,0.0);

In [8]:
@time interact!(t, 0.3, ax, ay, az, 1.0e-3);

In [9]:
@time collect!(t)

  2.941971 seconds (242.79 M allocations: 3.620 GB, 9.81% gc time)


In [10]:
@time accel!(t, ax,ay,az)

  0.302225 seconds (72.31 k allocations: 2.335 MB)


In [11]:
mean(abs(ax))/1e5

165.32256969711847

In [57]:
mx,myd,e50,e90,e95,e99,m,s = perf(particles, ax,ay,az, 10000, 1.0e-3)
print(myd)

mean(abs(rax)) = 508432.1386244696
[0.0,0.0,1.0,0.0,0.0,4.0,11.0,23.0,52.0,101.0,220.0,458.0,847.0,1486.0,1931.0,1919.0,1438.0,831.0,402.0,153.0,64.0,31.0,18.0,5.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0]

In [58]:
semilogx(10.^mx,myd,"-k")

LoadError: UndefVarError: semilogx not defined

In [59]:
e50

0.5709433750039612

In [60]:
e90

1.1480828500562679

In [61]:
e99

2.4839207584401035

In [17]:
function doall(t, ax,ay,az, alpha, eps2)
    total_mass = t.trees[1].total_mass
    for it in t.trees
        fill!(it.exps, NodeExp());
        it.total_mass = total_mass
    end
    tic()
    group!(t)
    t_group = toq()
    tic()
    inform!(t)
    t_inf = toq()
    fill!(ax,0.0)
    fill!(ay,0.0)
    fill!(az,0.0)
    tic()
    interact!(t, alpha, ax, ay, az, eps2)
    t_int = toq()
    tic()
    collect!(t)
    t_coll = toq()
    tic()
    accel!(t,ax,ay,az)
    t_acc = toq()
    t_group, t_inf, t_int, t_coll, t_acc
end

doall (generic function with 1 method)

In [ ]:
s_g = Float64[]
s_inf = Float64[]
s_int = Float64[]
s_coll = Float64[]
s_acc = Float64[]
s_all = Float64[]
for i in 1:20
    tic()
    t_group, t_inf, t_int, t_coll, t_acc = doall(t, ax, ay, az, 0.3, 1.0e-3);
    t_all = toq()
    append!(s_g, t_group)
    append!(s_inf, t_inf)
    append!(s_int, t_int)
    append!(s_coll, t_coll)
    append!(s_acc, t_acc)
    append!(s_all, t_all)
    @show i
end

i = 1
i = 

In [ ]:
@show mean(s_g)
@show mean(s_inf)
@show mean(s_int)
@show mean(s_coll)
@show mean(s_acc)
@show mean(s_all);

In [16]:
@show mean(s_g)
@show mean(s_inf)
@show mean(s_int)
@show mean(s_coll)
@show mean(s_acc)
@show mean(s_all);

mean(s_g) = 1.3853150495000002
mean(s_inf) = 0.7206255572
mean(s_int) = 1.8021083284
mean(s_coll) = 2.9465398244
mean(s_acc) = 0.24285650790000002
mean(s_all) = 7.644699567650001


In [10]:
t.trees[1].num_nodes_used / length(t.trees[1].nodes)

0.6026886456755053

In [11]:
sizeof(t.tree.nodes)/2^30

1.700000025331974

In [12]:
sizeof(t.tree.particles)/2^30

3.725290298461914e-8

In [13]:
sizeof(t.tree.exps)/2^30

1.9000000283122063

In [14]:
sizeof(ax)*4*2/2^30

2.0

In [15]:
1.7+3.75+1.9+2

9.35

In [16]:
64/9.35*2^25/2^20

219.0374331550802